In [1]:
!pip install -q langchain
!pip install -q torch
!pip install -q transformers
!pip install -q sentence-transformers
!pip install -q faiss-cpu
!pip install -q langchain-community
!pip install -qU langchain_core
!pip install -q tiktoken pypdf rapidocr-onnxruntime
!pip install -q transformers einops accelerate bitsandbytes
!pip install sentence_transformers
!pip install pdfplumber


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

In [4]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.8 MB/s eta 0:00:00


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import pdfplumber
from langchain.schema import Document
from langchain.prompts import ChatPromptTemplate


In [6]:
with pdfplumber.open('/content/RAG.pdf') as pdf:
  text = ""
  for page in pdf.pages:
    page = page.extract_text()
    text +=page

In [14]:

docs = [Document(page_content=text)]


In [15]:
# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
    # separators=["\n#{1,6} ",
    # "```\n",
    # "\n\\*\\*\\*+\n",
    # "\n---+\n",
    # "\n___+\n",
    # "\n\n",
    # "\n",
    # " ",
    # "",]
)

# Split the documents into smaller chunks
splitted_docs = text_splitter.split_documents(docs)


In [16]:
# Load embeddings using HuggingFace
# import torch
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"},  # Set the device to 'cuda' if gpu is present
    encode_kwargs={"normalize_embeddings": False}
)

# Create a FAISS vector database
vector_db = FAISS.from_documents(splitted_docs, embeddings)


<ipython-input-16-86b26d97a92f>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [17]:
# Perform a similarity search
query_result = vector_db.similarity_search("What is RAG?", k=3)

# Print the first result
print(query_result[0].page_content)


to the user’s query. These articles, combined with the original
current evaluation benchmarks and tools. Additionally,
question, form a comprehensive prompt that empowers LLMs
we anticipate future directions for RAG, emphasizing
to generate a well-informed answer.
potential enhancements to tackle current challenges.
The RAG research paradigm is continuously evolving, and
we categorize it into three stages: Naive RAG, Advanced
The paper unfolds as follows: Section II introduces the
RAG, and Modular RAG, as showed in Figure 3. Despite
main concept and current paradigms of RAG. The following
RAG method are cost-effective and surpass the performance
three sections explore core components—“Retrieval”, “Gen-
of the native LLM, they also exhibit several limitations.
eration”and“Augmentation”,respectively.SectionIIIfocuses
The development of Advanced RAG and Modular RAG is
onoptimizationmethodsinretrieval,includingindexing,query
a response to these specific shortcomings in Naive RAG.


In [20]:
template = """ You are an assistant for question-answering tasks.
  Use the following pieces of retrieved context to answer the question.
  Use ten sentences maximum and keep the answer concise.
  Question: {question}
  Context: {context}
  Answer:
"""
prompt = ChatPromptTemplate.from_template(template)


In [40]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, pipeline
from langchain.llms import HuggingFacePipeline
from transformers import BitsAndBytesConfig

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("deepset/roberta-base-squad2", padding=True, truncation=True, max_length=512)

# Configure 8-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Enable 8-bit loading
    llm_int8_threshold=6.0,  # Default threshold for mixed precision
    llm_int8_enable_fp32_cpu_offload=True  # Offload FP32 operations to CPU
)
# mistralai/Mistral-7B-Instruct-v0.1
# deepset/roberta-base-squad2
# Load the model in 8-bit
model = AutoModelForQuestionAnswering.from_pretrained(
    "deepset/roberta-base-squad2",
    device_map="auto",  # Automatically map model layers to devices (CPU/GPU)
    # quantization_config=bnb_config
)

# Create a pipeline for question answering
question_answer = pipeline(
    "question-answering",
    model=model,
    tokenizer=tokenizer,
    return_tensors="pt",
    max_new_tokens=512,
    temperatur=0.7,
    top_p=0.9
)

# Wrap the pipeline into HuggingFacePipeline for LangChain
llm = HuggingFacePipeline(
    pipeline=question_answer,
    model_kwargs={"temperature": 0.7, "max_length": 512}
)

# Example usage
print("Model loaded successfully in 8-bit!")


Device set to use cpu


Model loaded successfully in 8-bit!


In [41]:
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [42]:
while True:
  user = input("Enter your question: ")
  if user.lower() in ['exit','bye','goodbye']:
    print("Goodbye! 👋")
    break
  retrieved_docs = retriever.get_relevant_documents(user)
  context = " ".join([doc.page_content for doc in retrieved_docs])

    # 🔹 Ensure context is not empty
  if not context.strip():
      print("No relevant information found. Please try another question.")
      continue
  result = question_answer({'question': user,"context":context},max_answer_length=150)
  print(f"Answer: {result['answer']}\n")

Enter your question: what is RAG?


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Answer: state-of-the-art

Enter your question: what does the rag do.


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Answer: Section II introduces the
RAG

Enter your question: what is the work of rag.


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Answer: Largelanguagemodel

Enter your question: exit
Goodbye! 👋
